In [2]:
import numpy as np
import sys
sys.path.append('/home/yungdankblast/Desktop/Research/delay-optimizer')
from DelayEffects.Analyzer_new import FuncOpt
from Optimizer_Scripts.DelayTypeGenerators import *
import os
from matplotlib import pyplot as plt
import pandas as pd

In [3]:
# Initialize
loss_name = 'Ackley'
dimensions = [2, 10, 100, 1000]
ack = {d: FuncOpt(loss_name, d) for d in dimensions}

In [4]:
# Read params
params = pd.read_csv("../hyperparameter_data.csv")

## Helper Functions

In [5]:
def parse(func_obj, delay_type, **kwargs):
    parsed = params.copy()
    parsed = parsed[parsed['loss_name'] == func_obj.loss_name]
    parsed = parsed[parsed['dim'] == func_obj.n]
    parsed = parsed[parsed['optimizer_name'] == func_obj.optimizer_name]
    parsed = parsed[parsed['maxiter'] == func_obj.maxiter]
    parsed = parsed[parsed['delay_type'] == delay_type.name]
    parsed = parsed[parsed['num_delays'] == delay_type.num_delays]
    if delay_type.name != 'undelayed':
        parsed = parsed[parsed['max_L'] == delay_type.max_L]
    for arg, val in kwargs.items():
        parsed = parsed[parsed[arg] == val]
    return parsed.sort_values('best_loss')

def validate(func_obj, parsed, delay_type, num_points=500):
    # Extract params
    lrs = parsed.loc[:,'learning_rate'].values
    lrs = np.hstack(([1.0], lrs))        # Add control params
    
    # Initialize points and optimize for each lr
    func_obj.initialize_points('random', num_points)
    mean_losses = []
    for i, lr in enumerate(lrs):
        func_obj.optimize(delay_type, param_type='given', const_lr=True, save_state=False, save_loss=True, 
                          save_grad=False, save_iters=False, param_file=None, learning_rate=lr)
        mean_losses.append(func_obj.get_mean_final(func_obj.loss_vals))
        
    ind = np.argmin(mean_losses)
    return lrs, ind, mean_losses[ind]

def multi_validate(func_obj, delay_type, num_points, num_tests, **kwargs):
    # Parse parameters
    parsed = parse(func_obj, delay_type, **kwargs)
    
    lr_dict = {}
    losses = []
    for j in range(num_tests):
        lrs, i, best_loss = validate(func_obj, parsed, delay_type, num_points)
        if lrs[i] in lr_dict:
            lr_dict[lrs[i]] += 1
        else:
            lr_dict[lrs[i]] = 1
        losses.append((lrs[i], best_loss))

    return lr_dict, losses

In [6]:
num_tests = 5
num_points = 1000

# Undelayed

In [7]:
delay_type = Undelayed()

In [8]:
func_obj = ack[2]
multi_validate(func_obj, delay_type, num_points, num_tests)

({0.922737113926347: 2, 1.475254062393599: 2, 1.449149755592396: 1},
 [(0.922737113926347, 17.485756384628164),
  (0.922737113926347, 17.265773579561305),
  (1.475254062393599, 17.802955810375284),
  (1.449149755592396, 17.39171695150381),
  (1.475254062393599, 17.60225185664381)])

In [9]:
func_obj = ack[10]
multi_validate(func_obj, delay_type, num_points, num_tests)

({0.470409016134651: 5},
 [(0.470409016134651, 19.44193822600018),
  (0.470409016134651, 19.439993632882935),
  (0.470409016134651, 19.426600093708117),
  (0.470409016134651, 19.430532957105576),
  (0.470409016134651, 19.460550175581506)])

In [10]:
func_obj = ack[100]
multi_validate(func_obj, delay_type, num_points, num_tests)

({0.001475358781431: 5},
 [(0.001475358781431, 19.53654184420885),
  (0.001475358781431, 19.538337906540672),
  (0.001475358781431, 19.53404196170198),
  (0.001475358781431, 19.53835485475633),
  (0.001475358781431, 19.542351085174502)])

In [11]:
func_obj = ack[1000]
multi_validate(func_obj, delay_type, num_points, num_tests)

KeyboardInterrupt: 

# Unform Delay, $L=1$

In [ ]:
delay_type = Uniform(max_L=1, num_delays=1000)

In [ ]:
func_obj = ack[2]
multi_validate(func_obj, delay_type, num_points, num_tests)

In [ ]:
func_obj = ack[10]
multi_validate(func_obj, delay_type, num_points, num_tests)

In [ ]:
func_obj = ack[100]
multi_validate(func_obj, delay_type, num_points, num_tests)

In [ ]:
func_obj = ack[1000]
multi_validate(func_obj, delay_type, num_points, num_tests)

In [ ]:
delay_type = Uniform(max_L=1, num_delays=2000)

In [ ]:
func_obj = ack[2]
multi_validate(func_obj, delay_type, num_points, num_tests)

In [ ]:
func_obj = ack[10]
multi_validate(func_obj, delay_type, num_points, num_tests)

In [ ]:
func_obj = ack[100]
multi_validate(func_obj, delay_type, num_points, num_tests)

In [ ]:
func_obj = ack[1000]
multi_validate(func_obj, delay_type, num_points, num_tests)

# Stochastic Delay, $L=1$

In [22]:
delay_type = Stochastic(max_L=1, num_delays=1000)

In [23]:
func_obj = ack[2]
multi_validate(func_obj, delay_type, num_points, num_tests)

({1.866778249548995: 4, 1.422374042939696: 1},
 [(1.866778249548995, 9018.621156779209),
  (1.866778249548995, 8519.022326653578),
  (1.866778249548995, 8059.9071123022695),
  (1.866778249548995, 7495.675581154205),
  (1.422374042939696, 9788.789231353461)])

In [24]:
func_obj = ack[10]
multi_validate(func_obj, delay_type, num_points, num_tests)

({1.0: 3, 0.498393887722313: 2},
 [(1.0, 200.63812213474483),
  (0.498393887722313, 11.347220194192822),
  (0.498393887722313, 12.317292845442633),
  (1.0, 6.33579508025069),
  (1.0, 6.468896758643433)])

In [25]:
func_obj = ack[100]
multi_validate(func_obj, delay_type, num_points, num_tests)

KeyboardInterrupt: 

In [ ]:
func_obj = ack[1000]
multi_validate(func_obj, delay_type, num_points, num_tests)

In [ ]:
delay_type = Stochastic(max_L=1, num_delays=2000)

In [ ]:
func_obj = ack[2]
multi_validate(func_obj, delay_type, num_points, num_tests)

In [ ]:
func_obj = ack[10]
multi_validate(func_obj, delay_type, num_points, num_tests)

In [ ]:
func_obj = ack[100]
multi_validate(func_obj, delay_type, num_points, num_tests)

In [ ]:
func_obj = ack[1000]
multi_validate(func_obj, delay_type, num_points, num_tests)

In [18]:
params = np.array([    # Placeholder, these values have no meaning
    ['Ackley', 2, False, 0, 3.1561, 0.7792, 2300.0],     
    ['Ackley', 10, False, 0, 3.1838, 0.6166, 1300.0],    
    ['Ackley', 100, False, 0, 3.7281, 0.3693, 400.0],   
    ['Ackley', 1000, False, 0, 2.7389, 0.4849, 1400.0], 
    ['Ackley', 2, True, 1, 2.048, 0.8246, 300.0],        
    ['Ackley', 10, True, 1, 2.8954, 0.6782, 800.0],       
    ['Ackley', 100, True, 1, 1.9611, 0.7111, 800.0],   
    ['Ackley', 1000, True, 1, 3.603, 0.6938, 1300.0],    
])